## Note :
<h4>1. Pengembangan klasifikasi waktu sebagai berikut :</h4>

- Pagi (06.00 - 10.59)
- Siang (11.00 - 14.59)
- Sore (15.00 - 17.59)
- Malam (18.00 - 23.59)
- Dini Hari (00.00 - 03.59)
- Subuh (04.00 - 05.59)

In [1]:
import requests
import m3u8
import urllib3
import os
import shutil
import glob
from datetime import datetime
import time
import cv2
import numpy as np
import pandas as pd

from moviepy.editor import VideoFileClip, concatenate_videoclips
import subprocess

# Disable insecure request warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import gspread
import gspread_dataframe as gd
from google.oauth2.service_account import Credentials

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import random
from tensorflow import keras

In [3]:
def strip_end(text, suffix):
    if not text.endswith(suffix):
        return text
    return text[:len(text)-len(suffix)]

def download_file(url):
    local_filename = url.split('/')[-1]
    print(f'link download ts file: {url}')
    
    while True:
        response = requests.get(url, stream=True, verify=False)
        response.raise_for_status()
        
        # write response content to a file
        with open("video.ts", "wb") as f:
            f.write(response.content)
        
        # check the size of the downloaded file
        file_size = os.path.getsize("video.ts")
        if file_size < 10 * 1024:  # if file size is less than 10 KB
            print(f"Downloaded file is only {file_size} bytes. Retrying...")
            os.remove("video.ts")  # delete the previous downloaded file
            continue  # retry the download
        else:
            print(f"Downloaded file size: {file_size} bytes")
            break  # file size is OK, exit the loop
        
    cur_path = os.getcwd()
    ts_loc = cur_path + '/ts_files'
    
    if not os.path.exists(f'{ts_loc}'):
        os.makedirs(f'{ts_loc}')
        
    f = open(f"{ts_loc}/{local_filename}", "wb")
    f.write(response.content)
    
    return local_filename

def mp4_conversion(file_path):
    infile = file_path
    outfile = file_path.replace('.ts','.mp4')
    
    pre_name = os.path.basename(file_path)
    pos_name = pre_name.replace('.ts','.mp4')
    
    clip = VideoFileClip(infile)
    clip.write_videofile(outfile, fps=20)
    clip.close()
    
    del clip
    
    print(f'######### {pre_name} converted into {pos_name} #########')
    
def getFrame(sec, count):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        if 6 <= jam_kejadian < 10:
            cv2.imwrite(f"{folder_daerah_gambar}/1. Pagi/{daerah.lower()}_{waktu_now}_{count}.jpg",image)
        elif 11 <= jam_kejadian < 15:
            cv2.imwrite(f"{folder_daerah_gambar}/2. Siang/{daerah.lower()}_{waktu_now}_{count}.jpg",image)
        elif 15 <= jam_kejadian < 18:
            cv2.imwrite(f"{folder_daerah_gambar}/3. Sore/{daerah.lower()}_{waktu_now}_{count}.jpg",image)
        elif 18 <= jam_kejadian < 24:
            cv2.imwrite(f"{folder_daerah_gambar}/4. Malam/{daerah.lower()}_{waktu_now}_{count}.jpg",image)
        elif 0 <= jam_kejadian < 4:
            cv2.imwrite(f"{folder_daerah_gambar}/5. Dini Hari/{daerah.lower()}_{waktu_now}_{count}.jpg",image)
        else :
            cv2.imwrite(f"{folder_daerah_gambar}/6. Subuh/{daerah.lower()}_{waktu_now}_{count}.jpg",image)
            
    return hasFrames

In [27]:
m3u8_urls = [
    {'url': "https://atcs-dishub.bandung.go.id:1990/Kopo/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/Kopo/' ,'region' : "SP Soekarno Hatta - Kopo", 'name': 'Kopo'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/GedebageUtara/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/GedebageUtara/' ,'region' : "SP Soekarno Hatta - Gedebage", 'name': 'Gedebage Arah Utara'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/GedebageSel/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/GedebageSelatan/' ,'region' : "SP Soekarno Hatta - Gedebage", 'name': 'Gedebage Arah Selatan'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/Buahbatu/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/Buahbatu/' ,'region' : "SP Soekarno Hatta - Buahbatu", 'name': 'Buahbatu'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/Batununggal/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/Batununggal/' ,'region' : "SP Soekarno Hatta - Batununggal", 'name': 'Batununggal'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/MochToha/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/MochToha/' ,'region' : "SP Soekarno Hatta - Moch. Toha", 'name': 'Moch. Toha'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/IstanaPlaza/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/IstanaPlaza/' ,'region' : "PTZ SP. Istana Plaza", 'name': 'Istana Plaza'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/CihampelasUtara/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/CihampelasUtara/' ,'region' : "PTZ SP. Istana Plaza", 'name': 'Cihampelas Utara'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/laswiut/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/laswiut/' ,'region' : "Laswi", 'name': 'Jalan Laswi'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/OtistaLingkar/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/OtistaLingkar/' ,'region' : "Otista", 'name': 'Otista Lingkar'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/PtInti/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/PtInti/' ,'region' : "PT Inti", 'name': 'PT Inti'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/MRamdan/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/MRamdan/' ,'region' : "M Ramdan", 'name': 'M Ramdan'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/RamdanBarat/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/RamdanBarat/' ,'region' : "M Ramdan", 'name': 'M Ramdan Barat'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/Sriwijaya/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/Sriwijaya/' ,'region' : "Sriwijaya", 'name': 'Sriwijaya'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/BuahBatuLingkar/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/BuahBatuLingkar/' ,'region' : "Buah Batu Lingkar", 'name': 'Lingkar Buah Batu'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/Martanegara/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/Martanegara/' ,'region' : "Martanegara", 'name': 'Martanegara'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/TalagaBodas/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/TalagaBodas/' ,'region' : "Talaga Bodas", 'name': 'Talaga Bodas'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/GatsuLingkar/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/GatsuLingkar/' ,'region' : "Gatsu Lingkar", 'name': 'Gatsu Lingkar'}
]

In [5]:
# DEEP LEARNING DETAILS

FAST_RUN = False
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
batch_size=15

# model = False
model = keras.models.load_model('C:/Users/GE00000668/Documents/hooka_old/cctv-rain-dry-image-classification-main/model_hujan.h5')
test_gen = ImageDataGenerator(rescale=1./255)

In [26]:
waktu_now = datetime.today().strftime("%Y-%m-%d-%H")
jam_kejadian = int(waktu_now[-2:])

wadah_df = pd.DataFrame({
    "waktu_prediksi": []
    , "daerah" : []
    , "prediksi": []
})

# Set up Google Sheets API credentials
sa = gspread.service_account(filename = f"{os.getcwd()}/bandung-cctv-mining-8f0fb1f6ae05.json")
client = sa.open('cctv-data-mining')

for link in m3u8_urls[-3:]:
    print(f"########## Akan download cctv daerah : {link['name']}")
    
    url_link = link['url']
    
    max_retries = 3
    retry_wait_time = 5
    
    for trial in range(max_retries):
        try:
            r = requests.get(url_link, verify=False)
            if r.status_code == 404:
                print(f"########## WARNING Daerah : {link['name']} tidak bisa diakses")
                continue
            m3u8_master = m3u8.loads(r.text)

            # print statement can be deleted, they were placed prior to debugging purposes.
            ts_path = link['ts_path']
            print('sudah mau masuk iterasi')
            for seg in m3u8_master.data['segments']:
                url = ts_path + seg['uri']
                print(f'downloading {seg["uri"]}')
                download_file(url)  #download_file is a self made function
                
        except :
            print(f"HTTP error occurred, retrying in {retry_wait_time} seconds...")
            time.sleep(retry_wait_time)
    
    cur_path = os.getcwd()
    ts_loc = cur_path + '/ts_files'
    
    ts_files = glob.glob(f"{ts_loc}/*.ts")
    ts_files.sort()
    
    for ts in ts_files:
        try:
            mp4_conversion(ts)
            print('berhasil konversi ke mp4')
        except:
            print(f'cannot convert {os.path.basename(ts)}')
    
    mp4_files = glob.glob(f"{ts_loc}/*.mp4")
    mp4_files.sort()
    
    wadah_video = []
    for vid in mp4_files:
        func_vid = VideoFileClip(vid)
        wadah_video += [func_vid]

    # Concat them
    try:
        final = concatenate_videoclips(wadah_video)
        daerah = link['name']
        final_name = f'{daerah}_{waktu_now}.mp4'
    except:
        print(f"Gagal dapat CCTV {link['name']}")
        continue
    
    # Write output to the file
    output_path = f"{cur_path}/output_file"
    namadaerah = f"{link['name']}"
    folder_daerah = f"{output_path}/{namadaerah}"
    
    if not os.path.exists(f'{folder_daerah}'):
        os.makedirs(f'{folder_daerah}')
    
    print(f'membuat file : {final_name}')
    final.write_videofile(f'{folder_daerah}/{final_name}')
    
    for vid in mp4_files:
        try:
            os.remove(f"{vid}")
        except:
            print(f'cannot delete yet')
    
    for ts in glob.glob(f"{ts_loc}/*.ts"):
        try:
            os.unlink(ts)
            os.remove(f"{ts}")
            print('berhasil hapus ts file')
        except:
            pass
        
        
    print("------------ Video to Frame/Image Conversion ------------")
    
    video_path = f'{folder_daerah}/{final_name}'
    output_pic = f"{cur_path}/output_pic"
    folder_daerah_gambar = f"{output_pic}/{daerah}"

    if not os.path.exists(f'{folder_daerah_gambar}'):
        os.makedirs(f'{folder_daerah_gambar}')
        os.makedirs(f'{folder_daerah_gambar}/1. Pagi')
        os.makedirs(f'{folder_daerah_gambar}/2. Siang')
        os.makedirs(f'{folder_daerah_gambar}/3. Sore')
        os.makedirs(f'{folder_daerah_gambar}/4. Malam')
        os.makedirs(f'{folder_daerah_gambar}/5. Dini Hari')
        os.makedirs(f'{folder_daerah_gambar}/6. Subuh')

    vidcap = cv2.VideoCapture(video_path)
    
    sec = 0
    frameRate = 1 #//it will capture image in each 1 second

    count = 1

    if 6 <= jam_kejadian < 10:
        last_path_pic = f"{folder_daerah_gambar}/1. Pagi"
        while os.path.exists(f"{folder_daerah_gambar}/1. Pagi/{daerah.lower()}_{waktu_now}_{count}.jpg"):
            count += 1
    elif 11 <= jam_kejadian < 15:
        last_path_pic = f"{folder_daerah_gambar}/2. Siang"
        while os.path.exists(f"{folder_daerah_gambar}/2. Siang/{daerah.lower()}_{waktu_now}_{count}.jpg"):
            count += 1
    elif 15 <= jam_kejadian < 18:
        last_path_pic = f"{folder_daerah_gambar}/3. Sore"
        while os.path.exists(f"{folder_daerah_gambar}/3. Sore/{daerah.lower()}_{waktu_now}_{count}.jpg"):
            count += 1
    elif 18 <= jam_kejadian < 24:
        last_path_pic = f"{folder_daerah_gambar}/4. Malam"
        while os.path.exists(f"{folder_daerah_gambar}/4. Malam/{daerah.lower()}_{waktu_now}_{count}.jpg"):
            count += 1
    elif 0 <= jam_kejadian < 4:
        last_path_pic = f"{folder_daerah_gambar}/5. Dini Hari"
        while os.path.exists(f"{folder_daerah_gambar}/5. Dini Hari/{daerah.lower()}_{waktu_now}_{count}.jpg"):
            count += 1
    else :
        last_path_pic = f"{folder_daerah_gambar}/6. Subuh"
        while os.path.exists(f"{folder_daerah_gambar}/6. Subuh/{daerah.lower()}_{waktu_now}_{count}.jpg"):
            count += 1

    success = getFrame(sec, count)
    initial = count

    while success:
        count += 1
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec, count)

    vidcap.release()

    try:
        os.remove(f"{video_path}")
        print('berhasil hapus video file')
    except:
        pass

    #     PREDIKSI
    print("------------ Prediction Phase ------------")
    
    test_filenames = os.listdir(f"{last_path_pic}")
    test_df = pd.DataFrame({
        'filename': test_filenames
    })
    nb_samples = test_df.shape[0]

    test_generator = test_gen.flow_from_dataframe(
        test_df, 
        last_path_pic, 
        x_col='filename',
        y_col=None,
        class_mode=None,
        target_size=IMAGE_SIZE,
        batch_size=batch_size,
        shuffle=False
    )
        
    predict = model.predict(test_generator, steps=np.ceil(nb_samples/batch_size))
    test_df['category_pred'] = np.argmax(predict, axis=-1)

    label_map = {0: 'dry', 1: 'rain'}
    test_df['category_pred'] = test_df['category_pred'].replace(label_map)
    test_df['category_pred'] = test_df['category_pred'].replace({ 'rain': 1, 'dry': 0 })

    input_df = pd.DataFrame({
        "waktu_prediksi": [waktu_now]
        , "daerah" : [namadaerah]
        , "prediksi": np.round(np.average(test_df["category_pred"].values))
    })

    wadah_df = wadah_df.append(input_df)
    print(f"########## Selesai prediksi daerah : {namadaerah}")

# Open the Google Sheet
sheet = client.sheet1

# Get the existing data from the Google Sheet
data = sheet.get_all_values()

# Convert the data to a pandas DataFrame
existing_df = pd.DataFrame(data[1:], columns=data[0])

# Identify non-existing rows based on the key columns
existing_keys = set(existing_df[['waktu_prediksi', 'daerah']].apply(tuple, axis=1))
non_existing_rows = wadah_df[~wadah_df[['waktu_prediksi', 'daerah']].apply(tuple, axis=1).isin(existing_keys)]

# Insert non-existing rows into the Google Sheet
for i, row in non_existing_rows.iterrows():
    sheet.append_row(row.values.tolist())

########## Akan download cctv daerah : Martanegara
sudah mau masuk iterasi
downloading seg21348.ts
link download ts file: https://atcs-dishub.bandung.go.id:1990/Martanegara/seg21348.ts
Downloaded file size: 91180 bytes
downloading seg21349.ts
link download ts file: https://atcs-dishub.bandung.go.id:1990/Martanegara/seg21349.ts
Downloaded file size: 116748 bytes
downloading seg21350.ts
link download ts file: https://atcs-dishub.bandung.go.id:1990/Martanegara/seg21350.ts
Downloaded file size: 129720 bytes
downloading seg21351.ts
link download ts file: https://atcs-dishub.bandung.go.id:1990/Martanegara/seg21351.ts
Downloaded file size: 119944 bytes
downloading seg21352.ts
link download ts file: https://atcs-dishub.bandung.go.id:1990/Martanegara/seg21352.ts
Downloaded file size: 115432 bytes
downloading seg21353.ts
link download ts file: https://atcs-dishub.bandung.go.id:1990/Martanegara/seg21353.ts
Downloaded file size: 115808 bytes
downloading seg21354.ts
link download ts file: https://a

Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/ts_files\seg21348.mp4
######### seg21348.ts converted into seg21348.mp4 #########
berhasil konversi ke mp4
Moviepy - Building video C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/ts_files\seg21349.mp4.
Moviepy - Writing video C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/ts_files\seg21349.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/ts_files\seg21349.mp4
######### seg21349.ts converted into seg21349.mp4 #########
berhasil konversi ke mp4
Moviepy - Building video C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/ts_files\seg21350.mp4.
Moviepy - Writing video C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/ts_files\seg21350.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/ts_files\seg21350.mp4
######### seg21350.ts converted into seg21350.mp4 #########
berhasil konversi ke mp4
Moviepy - Building video C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/ts_files\seg21351.mp4.
Moviepy - Writing video C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/ts_files\seg21351.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/ts_files\seg21351.mp4
######### seg21351.ts converted into seg21351.mp4 #########
berhasil konversi ke mp4
Moviepy - Building video C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/ts_files\seg21352.mp4.
Moviepy - Writing video C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/ts_files\seg21352.mp4



...

Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/ts_files\seg15100.mp4
######### seg15100.ts converted into seg15100.mp4 #########
berhasil konversi ke mp4
Moviepy - Building video C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/ts_files\seg15101.mp4.
Moviepy - Writing video C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/ts_files\seg15101.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/ts_files\seg15101.mp4
######### seg15101.ts converted into seg15101.mp4 #########
berhasil konversi ke mp4
membuat file : Talaga Bodas_2023-05-18-16.mp4
Moviepy - Building video C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/output_file/Talaga Bodas/Talaga Bodas_2023-05-18-16.mp4.
Moviepy - Writing video C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/output_file/Talaga Bodas/Talaga Bodas_2023-05-18-16.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/output_file/Talaga Bodas/Talaga Bodas_2023-05-18-16.mp4
------------ Video to Frame/Image Conversion ------------
berhasil hapus video file
------------ Prediction Phase ------------
Found 27 validated image filenames.
2/2 [==============================] - 0s 71ms/step
########## Selesai prediksi daerah : Talaga Bodas
########## Akan download cctv daerah : Gatsu Lingkar
sudah mau masuk iterasi
downloading seg0.ts
link download ts file: https://atcs-dishub.bandung.go.id:1990/GatsuLingkar/seg0.ts
Downloaded file size: 42864 bytes
sudah mau masuk iterasi
downloading seg0.ts
link download ts file: https://atcs-dishub.bandung.go.id:1990/GatsuLingkar/seg0.ts
Downloaded file size: 42864 bytes
sudah mau masuk iterasi
downloading seg0.ts
link download ts file: https://atcs-dishub.bandung.go.id:1990/GatsuLingkar/seg0.ts
Downloaded file size: 42864 bytes
Moviepy - Building video C:\Users\GE00000668\Docum

Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/ts_files\seg0.mp4
######### seg0.ts converted into seg0.mp4 #########
berhasil konversi ke mp4
membuat file : Gatsu Lingkar_2023-05-18-16.mp4
Moviepy - Building video C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/output_file/Gatsu Lingkar/Gatsu Lingkar_2023-05-18-16.mp4.
Moviepy - Writing video C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/output_file/Gatsu Lingkar/Gatsu Lingkar_2023-05-18-16.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping\cctv-indonesia/output_file/Gatsu Lingkar/Gatsu Lingkar_2023-05-18-16.mp4
------------ Video to Frame/Image Conversion ------------
berhasil hapus video file
------------ Prediction Phase ------------
Found 4 validated image filenames.
1/1 [==============================] - 0s 100ms/step
########## Selesai prediksi daerah : Gatsu Lingkar
